<a href="https://colab.research.google.com/github/karthikkodakandla/dataset/blob/master/Assignment2_karthik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import ResNet50

print(tf.__version__)

2.3.0


In [2]:
from google.colab import drive
drive.mount("/content/drive")
"4/4gGieupuu9nlrt02jqlN2V25fX1C5GFUct7zXuezrKtHLgmzYS6Tyl0"

Mounted at /content/drive


'4/4gGieupuu9nlrt02jqlN2V25fX1C5GFUct7zXuezrKtHLgmzYS6Tyl0'

In [3]:
col_names = ['image','label']
df = pd.read_csv('/content/drive/My Drive/Flower_images/Flower_image/classlabels.txt', header =None, names = col_names)
df = df.sort_values(by='image')
df

,image,label
0,JFT_00001.jpg,1
1,JFT_00002.jpg,1
2,JFT_00003.jpg,1
3,JFT_00004.jpg,1
558,JFT_00005.jpg,1
...,...,...
555,JFT_01475.jpg,30
1477,JFT_01476.jpg,30
1478,JFT_01477.jpg,30
556,JFT_01478.jpg,30


In [4]:
df['label'].value_counts()

6     70
28    68
23    67
13    66
4     66
2     65
7     64
18    63
29    63
20    61
27    61
24    60
10    60
25    56
19    55
14    54
5     54
22    49
12    48
9     46
17    45
3     41
21    38
11    36
26    35
8     28
16    24
15    13
30    12
1     11
Name: label, dtype: int64

In [5]:
import os
directory ='/content/drive/My Drive/Flower_images/Flower_image/jpg/'
images = sorted(os.listdir(directory))

In [6]:
len(images)

1479

In [7]:
x_train = []
import time
from progressbar import ProgressBar
pbar = ProgressBar()

for imgs in pbar(images):
  img_path = os.path.join(directory, imgs)
  img = load_img(img_path, color_mode = 'rgb', target_size = (32, 32))
  img = img_to_array(img).astype('float32')/255
  x_train.append(img)
 



100% (1479 of 1479) |####################| Elapsed Time: 0:04:51 Time:  0:04:51


In [8]:
y=np.array(df['label'])

In [9]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
y=pd.DataFrame(enc.fit_transform(df[['label']]).toarray())


In [10]:
len(x_train)

1479

In [11]:
y.shape

(1479, 30)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y, test_size = 0.25)

In [13]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


In [17]:
NUM_CLASSES = 30
IMG_ROW,IMG_COL = 32,32
from keras.layers.advanced_activations import LeakyReLU
def resnet():
    # Build a Sequential Model
    model = Sequential()
    
    # Add Inception module
    model.add(ResNet50(include_top=False, input_shape=(IMG_ROW, IMG_COL, 3)))
    
    # Flatten *** Most Important *** Never forget to flatten a conv output before dense
    ## This is necessary for resolving dimension errors 
    model.add(Flatten()) 
    
    # Fully connected layer
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.02))
    
    # Fully connected layer
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.02))
    
    # Output Layer
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    
    # Set trainable false to use pretrained weights and not update them
    model.layers[0].trainable = False
    
    # Compile the model
    model.compile(optimizer = 'adam', loss ='categorical_crossentropy', metrics = ['accuracy'])
    
    return model
model = resnet()

94773248/94765736 [==============================] - 1s 0us/step


In [19]:
x_train = np.array(x_train)

In [20]:
# Define training params
batch_size = 164
epochs = 10 # Set to 1 for demo, for good results set 100, Running for 100 epochs will take time.
train_steps_per_epoch = x_train.shape[0] // batch_size


from keras.callbacks import ReduceLROnPlateau
red_lr= ReduceLROnPlateau(monitor='val_accuracy',patience=3,verbose=1,factor=0.1)

In [21]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images ideally true
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [22]:
from keras.optimizers import Adam
for layer in model.layers:
    layer.trainable = True
adam_fine = Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False) #20x smaller than standard
model.compile(optimizer=adam_fine, loss='categorical_crossentropy', metrics=['accuracy'])

#model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 30)               

In [24]:
x_test = np.array(x_test)


In [ ]:
# Train the model



In [ ]:
History = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 1,steps_per_epoch=x_train.shape[0] // batch_size)
# model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size,validation_data = (x_test,y_test))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
6/6 [==============================] - 61s 10s/step - loss: 3.7930 - accuracy: 0.0640 - val_loss: 3.9856 - val_accuracy: 0.0432
Epoch 2/10
3/6 [==============>...............] - ETA: 23s - loss: 3.4209 - accuracy: 0.0894

In [ ]:
def plot_history(history):
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.plot(history.epoch, np.array(history.history['accuracy']),
           label='Train Loss')
  plt.plot(history.epoch, np.array(history.history['val_accuracy']),
           label = 'Val loss')
  plt.legend()
  plt.ylim([0, 1])

plot_history(History)


In [ ]:
print(max(History.history['val_accuracy']))

In [ ]:
test_img_10 = plt.imread("/content/class_10.jpg") 
test_img_28 = plt.imread("/content/class_28.jpg")
test_img_29 = plt.imread('/content/class_29.jpg')
test_img = plt.imread('/content/fly orchid 1.jpg')
plt.imshow(test_img)



In [ ]:
from skimage.transform import resize
resized_image = resize(test_img, (32,32,3))
predictions = model.predict(np.array( [resized_image] ))



In [ ]:
predictions.argmax(axis=1)